## 용어 정의 

* DSM : digital surface model의 약자이며 tiff 포멧의 이미지 파일입니다.   
    *  Pixel 구성요소 : 높이 값 (float type Z value)  
    *  Shape : 2차원 (x,y)   
* 정사영상(orthomosaic) : 현장 사진이며 tiff 포멧의 이미지 파일입니다.  
    * Pixel 구성요소 : 4개의 색상채널 Matrix [ R , G , B , A ]    
    * shape : 3차원 (x,y,4)  
  
  
> 드론이 하늘에서 찍은 사진들을 Stitching 해서 Geotiff 태그(지리정보)를 입힌것이 정사영상입니다.  
> DSM은 이 정사영상의 각 픽셀들에 대한 높이(Z)값을 담은 파일입니다.  
> 두 파일은 픽셀 구성요소와 매트릭스 차원 구성을 제외하고 모두 동일합니다.   
  
---
  
* 양수화 : 음수와 양수로 구성된 숫자 분포의 최소값이 0이 되도록 분포를 0 위로 올려서 음수가 없도록 하는것입니다.  
  
>  Min-Max Normalization 연산을 위해서 분포는 유지하되, 음수가 없도록 하는 트릭입니다.

## Surface2Color  
``` rgba_matrix = Surface2Color(dsm_matrix).transform() ```  
  
* 컬러맵은 256개의 요소로 이루어진 리스트입니다. 0~255 인덱스로 접근하며, 인덱스가 255에 가까울수록 높은 Z값을 표현해줍니다.  
1. DSM의 픽셀(Z값)에 곱하면 컬러맵 인덱스가 나오게 해주는 미지수 n을 구합니다. -> DSM_Pixel x n = Colormap Index 
2. 모든 픽셀에 대해서 Z값(DSM 픽셀)을 RGBA채널로 변환시키는 연산을 수행합니다. (셈플 파일의 경우 **1억 6930만 7930개의 픽셀**로 이루어져 있습니다.)
    * ```if z == 결측치: return (0,0,0,0) else: return Colormap[ round(z x n) ]```

In [ ]:
from typing import Iterable
from importlib import import_module

import numpy as np
import pandas as pd
from pandarallel import pandarallel

class Surface2Color:
    """DSM(digital surface model) 매트릭스를 PNG(rgba) 매트릭스로 변환합니다."""

    MISSING_VALUE = -10000 # z 
    FEATURE_RANGE = 255 # 0~255 (colormap max index)
    MISSING_PIXEL = (0,0,0,0) # rgba 
    COLOR_MAP = import_module("colormap").MATRIX 

    def __init__(self,matrix:Iterable):
        """ fit 함수를 겸합니다. """
        self._n = self._calculation_n(matrix)
        self.data_frame = pd.DataFrame(matrix)
    
    def transform(self) -> pd.DataFrame:
        """ Z DataFrame -> RGBA DataFrame """
        
        pandarallel.initialize(progress_bar=True) # progress_bar 옵션은 테스트시에만 사용
        rgba_df = self.data_frame.parallel_applymap(self._proc_unit)
        # reshape 작업 , pd.DataFrame 요소가 리스트라고 shape 처리가 되는것이 아니므로 수동으로 해야 한다
        rgba_flatten = np.array([list(rgba) for rgba in rgba_df.to_numpy().flatten()]).flatten()
        
        rgba_shape = self.data_frame.shape + (4,)
        return rgba_flatten.reshape(rgba_shape).astype(np.uint8)
    
    def _proc_unit(self,z:float) -> np.array:
        """ z 값을 rgba로 변환합니다. """
        if z == self.MISSING_VALUE:
            return self.MISSING_PIXEL
        if self._min_v < 0:
            z += abs(self._min_v)
        normalized_value = z * self._n
        return self.COLOR_MAP[round(normalized_value)]

    def _calculation_n(self,matrix) -> float:
        """ nomalization 을 위한 숫자 n을 구합니다 """

        values = list(np.unique(np.array(matrix).flatten()))
        values.remove(self.MISSING_VALUE)
        self._min_v , self._max_v = min(values), max(values)
        # 모든 값들을 양수로 올렸을 때 최대값
        self._pas_max_v = self._max_v + ( abs(self._min_v) if self._min_v < 0 else 0 )

        return self.FEATURE_RANGE / self._pas_max_v